In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np
!pip install folium
import folium

     |████████████████████████████████| 92kB 11.6MB/s eta 0:00:01


In [231]:
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_b42de251f40d4fb0af2b64da74315ba7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-67KSrY2W0iyDTqlDOBF8CMkC8z9c5SRqhqcucpQil4J',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_b42de251f40d4fb0af2b64da74315ba7.get_object(Bucket='ultimomodulo-donotdelete-pr-qt4yozsgmyysu2',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

cityloc = pd.read_csv(body)


### We first extract web page and table

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable sortable')

### Add the items to a df

In [5]:
rows = table.find_all('tr')
postalcodes_df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        info = [cell.text.split('\n')[0] for cell in cells]
        postalcodes_df.loc[len(postalcodes_df)] = info

### remove the not assigned

In [6]:
for c, row in enumerate(postalcodes_df['Borough']):
    if row == 'Not assigned':
        postalcodes_df.drop(c,inplace = True)
postalcodes_df = postalcodes_df.reset_index()
postalcodes_df.drop(['index'], axis = 1, inplace = True)

### the replace the / for ,

In [7]:
for c, row in enumerate(postalcodes_df['Neighborhood']):
    new_text = row.replace('/',',')
    postalcodes_df.at[c,'Neighborhood'] = new_text    

In [8]:
'Not assigned' in set(postalcodes_df)

False

In [10]:
postalcodes_df.shape

(103, 3)

In [9]:
postalcodes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## The geocoder wont work when loading all the data

i tried to use only the coordenates from the cities but where kind of bad, so after this i used the downloadable file

In [10]:
 from geopy.geocoders import Nominatim

In [11]:
geolocator = Nominatim( user_agent = 'coursera')


citylist = postalcodes_df['Borough'].unique().tolist()

citylistloc = []

for city in citylist:
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    information = [city,latitude,longitude]
    citylistloc.append(information)
    
citylistloc


[['North York', 43.7543263, -79.44911696639593],
 ['Downtown Toronto', 43.6541737, -79.38081164513409],
 ['Etobicoke', 43.671459150000004, -79.55249206611668],
 ['Scarborough', 54.2820009, -0.4011868],
 ['East York', 39.9737093, -76.6863554],
 ['York', 53.9590555, -1.0815361],
 ['East Toronto', 43.6534817, -79.3839347],
 ['West Toronto', 43.6534817, -79.3839347],
 ['Central Toronto', 43.6534817, -79.3839347],
 ['Mississauga', 43.590338, -79.645729]]

### So i used the downloadable data as cityloc

In [12]:
cityloc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### we combine the frames

In [13]:

for c,city in enumerate(postalcodes_df['PostalCode']):
    for d, code in enumerate(cityloc['Postal Code']):
        if city == code:
            postalcodes_df.at[c,'Lat'] = cityloc.at[d,'Latitude']
            postalcodes_df.at[c,'Lng'] = cityloc.at[d,'Longitude']

### Filter the ones with tornto on it

In [14]:
c=0
for c, Toronto in enumerate(postalcodes_df['Borough']):
    if ('Toronto' in Toronto)== False:
        postalcodes_df.drop(c,inplace=True)
        
postalcodes_df = postalcodes_df.reset_index(drop=True)

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent='coursera')
location = geolocator.geocode(address)
latitude2 = location.latitude
longitude2 = location.longitude

In [18]:
map_toronto = folium.Map(location = [latitude2,longitude2], zoom_start = 12)

for lat, lng, neigh, borough in zip(postalcodes_df['Lat'],postalcodes_df['Lng'],postalcodes_df['Neighborhood'],postalcodes_df['Borough']):
    label = f'{neigh},{borough}'
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html= False).add_to(map_toronto)
    
map_toronto
    

## start to play with the four square

In [239]:
CLIENT_ID = '2XC45W02YYRHDCWS3IFOONQ13MQDP5N1W5YCJ40MVWQKGWZ1'
CLIENT_SECRET = 'FDD3MIPRZBCJG2LDSE52VHFZ53OJSSETGKTJ5PHPRJUU1WKJ'
VERSION = '20180605'

LIMIT = 100
import requests

In [238]:
postalcodes_df.head()

,PostalCode,Borough,Neighborhood,Lat,Lng
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### We retrieve the information from the foursquare api, for each zipcode location

In [ ]:
def getnearvenues (names, latitudes,longitudes,radius = 500):
    
    venues_list= []
    for name, lat, lng in zip(names,latitudes,longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

### apply the funcs

In [184]:
toronto_venues = getnearvenues(names = postalcodes_df['Neighborhood'], latitudes = postalcodes_df['Lat'], longitudes = postalcodes_df['Lng'])

In [240]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [185]:
len(toronto_venues['Venue'].unique())

1045

### One hot code, add neighboorhood names and order columns

In [186]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

column_fixer = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[column_fixer]

In [187]:
toronto_onehot.groupby('Neighborhood').sum()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
"Brockton , Parkdale Village , Exhibition Place",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Business reply mail Processing CentrE,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",0,1,1,1,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Central Bay Street,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Christie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Church and Wellesley,2,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
"Commerce Court , Victoria Hotel",0,0,0,0,0,0,4,0,0,1,...,0,0,0,0,0,2,0,0,1,0
Davisville,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [237]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Business reply mail Processing CentrE,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.0


Search for top venues

In [180]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [223]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Beer Bar,Cocktail Bar,Seafood Restaurant,Farmers Market,Cheese Shop,Italian Restaurant,Café
1,"Brockton , Parkdale Village , Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Farmers Market
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Sculpture Garden,Airport
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Salad Place,Ice Cream Shop,Fried Chicken Joint


### Once sorted we cluster

In [191]:
from sklearn.cluster import KMeans

In [224]:
kclusters = 8
toronto_clustering = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = kclusters, random_state=0,).fit(toronto_clustering)
kmeans.labels_

array([6, 6, 0, 0, 6, 0, 6, 6, 6, 7, 0, 6, 5, 6, 6, 0, 6, 0, 4, 6, 2, 0,
       6, 6, 6, 6, 2, 1, 6, 6, 6, 6, 6, 6, 6, 3, 0, 6, 6], dtype=int32)

In [225]:
cluster_label = pd.DataFrame(kmeans.labels_)
neighborhoods_venues_sorted['Cluster'] = cluster_label

In [226]:
column_fixer2 = [neighborhoods_venues_sorted.columns[0]] + [neighborhoods_venues_sorted.columns[-1]] + list(neighborhoods_venues_sorted.columns[1:-1])
neighborhoods_venues_sorted = neighborhoods_venues_sorted[column_fixer2]

### Combine the first dat frame with clustered and sorted

In [229]:
toronto_join = postalcodes_df
toronto_join = toronto_join.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
toronto_join.head()

,PostalCode,Borough,Neighborhood,Lat,Lng,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,6,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,6,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,6,Clothing Store,Coffee Shop,Café,Restaurant,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Coffee Shop,Café,Hotel,Gastropub,American Restaurant,Cocktail Bar,Italian Restaurant,Seafood Restaurant,Cosmetics Shop,Department Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Trail,Pub,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## we mappp

In [236]:
map_clusters = folium.Map(location=[latitude2, longitude2], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_join['Lat'], toronto_join['Lng'], toronto_join['Neighborhood'], toronto_join['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# And finally we are done